<a href="https://colab.research.google.com/github/antoinexp/markov-chains-COM-516/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import scipy.stats as st
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [41]:
class DatasetGenerator(object):
  def __init__(self, N=100):
    self.N = N
    self.x = None
    self.v = None
    self.refresh()
  
  def refresh(self):
    raise Exception("undefined")

In [42]:
class G1(DatasetGenerator):
  def refresh(self):
    self.x = st.uniform().rvs((self.N,2))
    self.v = st.uniform().rvs((self.N,))

In [43]:
class G2(DatasetGenerator):
  def refresh(self):
    self.x = st.uniform().rvs((self.N,2))
    self.v = np.exp(st.norm(-0.85, 1.3).rvs((self.N,)))

In [44]:
g1 = G1()

In [46]:
class MCMCSolver: 
    def __init__(self, dataset,lmbd):
        self.dataset = dataset
        self.state = np.random.binomial(1, 0.5, dataset.N) #randomly initialize state
        self.lmbd = lmbd
        self.beta = 0.1
        
    #compute the radius of the state
    def radius(self,state):
        #TODO;
        return 0
    
    def compute_diff(self,state_1,state_2,i):
        diff = (state_1[i] - state_2[i])*self.dataset.v[i] \
        - self.lmbd*self.dataset.N*np.pi*(self.radius(state_1)-self.radius(state_2))
        return diff

    def acceptance_proba(self,diff,beta):
        proba = np.exp(beta*diff)
        return np.minimum(1.0,proba)
    
    def solve(self,n_it):
        print("Initial state:",self.state)
        for it in range(n_it):
            next_state = self.state.copy()
            
            #flip one state at random
            i = np.random.choice(self.dataset.N)
            next_state[i] = 1^next_state[i]
                        
            #update state with acceptance probability
            diff = self.compute_diff(self.state, next_state,i)
            if np.random.uniform() < self.acceptance_proba(diff, self.beta):
                self.state = next_state
                
            #beta scheduling    
            if (it % 1000) == 0:
                self.beta = self.beta       
        return self.state

In [47]:
mcmc = MCMCSolver(dataset = g1, lmbd = 1)
mcmc.solve(n_it = 2000)

Initial state: [0 1 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1
 1 0 1 0 1 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 0 0 0 1 0 0 1 0 1 0
 0 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 1 0 0 1 1 0 0]


array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0])